# Task B: Named Entity Recognition with CRF on Hindi Dataset. (Total: 60 Points out of 100)

In this part, you will use a CRF to implement a named entity recognition tagger.
We have implemented a CRF for you in crf.py along with some functions to build, and pad feature vectors. Your job is to add more features to learn a better tagger. Then you need to complete the traiing loop implementation.

Finally, you can checkout the code in `crf.py` -- reflect on CRFs and span tagging, and answer the discussion questions.


We will use the Hindi NER dataset at: https://github.com/cfiltnlp/HiNER

The first step would be to download the repo into your current folder of the Notebook

In [1]:
# !git clone https://github.com/cfiltnlp/HiNER.git

fatal: destination path 'HiNER' already exists and is not an empty directory.


In [1]:
import torch
import nltk

In [2]:
# This is so that you don't have to restart the kernel everytime you edit hmm.py

%load_ext autoreload
%autoreload 2

## First we load the data and labels. Feel free to explore them below.

Since we have provided a seperate train and dev split, there is not need to split the data yourself.

In [3]:
from crf import load_data, make_labels2i

train_filepath = "preamble_train.txt"
# dev_filepath = "./HiNER/data/collapsed/validation.conll"
labels_filepath = "all_labels.txt"

train_sents, train_tag_sents = load_data(train_filepath)
dev_sents, dev_tag_sents = train_sents[int(len(train_sents) * 9 // 10):], train_tag_sents[int(len(train_sents) * 9 // 10):]
train_sents, train_tag_sents = train_sents[:int(len(train_sents) * 9 // 10)], train_tag_sents[:int(len(train_sents) * 9 // 10)]
labels2i = make_labels2i(labels_filepath)

print("train sample", train_sents[2], train_tag_sents[2])
print()
print("labels2i", labels2i)

train sample ['Before', 'The', 'Madurai', 'Bench', 'Of', 'Madras', 'High', 'Court', 'Dated', ':', '23/12/2011', 'Coram', 'The', 'Honourable', 'Mr.', 'Justice', 'V.Ramasubramanian', 'Civil', 'Revision', 'Petition', '(', 'Npd)(Md', ')', 'No.1123', 'of', '2006', 'And', 'M.P.No.2', 'of', '2006', '1', '.', 'Ayisha', 'Beevi', '2', '.', 'Beevija', '3', '.', 'Hadijath', 'Beevi', '4', '.', 'Yunusa', 'Begam', '5', '.', 'Syed', 'Ali', '6', '.', 'Sumaya', 'Begam', '7', '.', 'Mohamed', 'Yoosuf', '8', '.', 'Mohamed', 'Ismail', '9', '.', 'Razira', 'Beevi', '10.Shabi', 'Mohamed', '11.Zakir', 'Mugain', '12.Ferosh', 'Khan', '13.Augustin', '14.Dr', '.', 'T.C.Joseph', '.....', 'Petitioners', 'Vs', '.', '1', '.', 'Sheik', 'Mydeen', '2', '.', 'A.P.Nelson', '3', '.', 'Chandrakala', 'Ruben', '.....', 'Respondents', '-----', 'Petition', 'under', 'Article', '227', 'of', ' ', 'the', 'Constitution', 'of', 'India', 'against', 'the', 'fair', 'and', 'decretal', 'order', 'dated', '18.10.2006', 'made', 'in', 'E.P.No.2

In [4]:
len(train_sents) + len(dev_sents)

1561

## Feature engineering. (Total 30 points)

Notice that we are **learning** features to some extent: we start with one unique feature for every possible word. You can refer to figure 8.15 in the textbook for some good baseline features to try.
![image.png](image2.png)

There is no need to worry about embeddings now.

### Hindi POS Tagger   (10 Points)

Although this step is not entirely necessary, if you want to use the HMM pos tagger to extract feature corresponding to the pos of the word in the sentence, we need to add this into the pipeline.

You get 10 points if you use your pos_tagger to featurize the sentences

In [5]:
from hmm import get_hindi_dataset
import pickle
from typing import List

words, tags, observation_dict, state_dict, all_observation_ids, all_state_ids = get_hindi_dataset()

# we need to add the id for unknown word (<unk>) in our observations vocab
UNK_TOKEN = '<unk>'

observation_dict[UNK_TOKEN] = len(observation_dict)
print("id of the <unk> token:", observation_dict[UNK_TOKEN])

## load the pos tagger 
pos_tagger = pickle.load(open('hindi_pos_tagger.pkl', 'rb'))

def encode(sentences: List[List[str]]) -> List[List[int]]:
    """
    Using the observation_dict, convert the tokens to ids
    unknown words take the id for UNK_TOKEN
    """
    return [
        [observation_dict[t] if t in observation_dict else observation_dict[UNK_TOKEN]
            for t in sentence]
        for sentence in sentences]

def get_pos(pos_tagger, sentences) -> List[List[str]]:
    """
    The the pos tag for input sentences
    """
    sentence_ids = encode(sentences)
    decoded_pos_ids = pos_tagger.decode(sentence_ids)
    return [
        [tags[i] for i in d_ids]
        for d_ids in decoded_pos_ids
    ]

id of the <unk> token: 2186


[nltk_data] Downloading package indian to /home/preetham/nltk_data...
[nltk_data]   Package indian is already up-to-date!


### Feature Engineering Functions (20 Points)

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
def get_sentences_tags(json_path):
    with open(json_path, 'r+') as f:
        json_data = json.load(f)

    sentences_tags = []
    for datapoint in tqdm(json_data):
        label_dicts = datapoint['annotations'][0]['result']
        sent = datapoint['data']['text']
        sent = sent.replace('\n', '\t')
        doc = nlp(sent)

        tag_list = []
        for label_dict in label_dicts:
            tag_list.append((label_dict['value']['start'], label_dict['value']['end'], label_dict['value']['labels'][0]))

        tokens = []
        tags = []
        pos = []
        for token in doc:
            tokens.append(token.text)
            if len(tag_list) == 0:
                tags.append('O')
                continue

            if token.idx > tag_list[0][1]:
                tag_list.pop(0)

            if len(tag_list) == 0:
                tags.append('O')
                continue

            if token.idx >= tag_list[0][0] and token.idx < tag_list[0][1]:
                if token.idx == tag_list[0][0]:
                    tags.append(f'B-{tag_list[0][2]}')
                else:
                    tags.append(f'I-{tag_list[0][2]}')
            else:
                tags.append('O')
            pos.append(token.pos_)
        sentences_tags.append([tokens, tags, pos])
    
    return sentences_tags

In [8]:
# sent_tag_pos = get_sentences_tags("NER_TRAIN/NER_TRAIN_JUDGEMENT.json")

In [9]:
# f = open("JUDGEMENT_BIO_POS.txt", "w")
# for sent, tag, pos in sent_tag_pos:
#     for s, t, p in zip(sent, tag, pos):
#         f.write(f"{s}\t{t}\t{p}")
#         f.write("\n")
    
#     f.write("\n")
# f.close()

In [10]:
# def process_data(sents, tags):
#     new_sents = []
#     new_tags = []
#     for sent, tag in zip(sents, tags):
#         tag_index = ['O']  * len(''.join(sent))

#         _sum = 0
#         for word, tag in zip(sent, tags):
#             for i in range(_sum, _sum + len(word)):
# #                 print(i, len(tag_index))
#                 tag_index[i] = tag
#             _sum += len(word)

#         spacy_tokens = []
#         spacy_tags = []
#         _sum = 0
#         for token in doc:
#             print(len(tag_index), _sum)
#             spacy_tokens.append(token.text)
#             spacy_tags.append(tag_index[_sum])
#             _sum += len(token.text)
    
#         new_sents.append(spacy_tokens)
#         new_tags.append(spacy_tags)
    
#     return new_sents, new_tags

# new_train_sents, new_train_tags = process_data(train_sents, train_tag_sents)

In [11]:
from typing import List
import spacy

# TODO: Update this function to add more features
#      You can check crf.py for how they are encoded, if interested.
def make_features(text: List[str]) -> List[List[int]]:
    """Turn a text into a feature vector.

    Args:
        text (List[str]): List of tokens.

    Returns:
        List[List[int]]: List of feature Lists.
    """
#     sent_tags = get_pos(pos_tagger, [text])[0]
    feature_lists = []
    # nlp = spacy.load("en_core_web_sm")
    # doc = nlp(' '.join(text))
    # doc = spacy.tokens.Doc(nlp.vocab, words=text)
    pos_tags = nltk.pos_tag(text)
    for i, token in enumerate(text):
        feats = []
        # We add a feature for each unigram.
        feats.append(f"word={token}")
        feats.append(f"prev_word={'<s>' if i == 0 else text[i - 1]}")
        feats.append(f"next_word={'</s>' if i == len(text) - 1 else text[i + 1]}")
        feats.append(1 if True in [i.isupper() for i in token] else 0)
        feats.append(1 if True in [i.isdigit() for i in token] else 0)
        feats.append(1 if token in '\t\n' else 0)
        feats.append(f"prefix={token[:3] if len(token) >= 3 else token}")
        feats.append(f"prefix={token[-3:] if len(token) >= 3 else token}")
        feats.append(len(token))
        # feats.append(pos_tags[i])
        # feats.append(token.is_stop)
        # TODO: Add more features here
#         feats.append(sent_tags[i - 1] if i != 0 else '<s>')
#         feats.append(sent_tags[i])
#         feats.append(sent_tags[i + 1] if i != len(text) - 1 else '</s>')
        # We append each feature to a List for the token.
        feature_lists.append(feats)

    return feature_lists

In [12]:
from tqdm import tqdm

def featurize(sents: List[List[str]]) -> List[List[List[str]]]:
    """Turn the sentences into feature Lists.
    
    Eg.: For an input of 1 sentence:
         [[['I','am','a','student','at','CU','Boulder']]]
        Return list of features for every token for every sentence like:
        [[
         ['word=I',  'prev_word=<S>','pos=PRON',...],
         ['word=an', 'prev_word=I'  , 'pos=VB' ,...],
         [...]
        ]]

    Args:
        sents (List[List[str]]): A List of sentences, which are Lists of tokens.

    Returns:
        List[List[List[str]]]: A List of sentences, which are Lists of feature Lists
    """
    feats = []
    for sent in sents:
        # Gets a List of Lists of feature strings
        feats.append(make_features(sent))

    return feats

## Finish the training loop.   (10 Points)

See the previous homework, and fill in the missing parts of the training loop.

In [13]:
from crf import f1_score, predict, PAD_SYMBOL, pad_features, pad_labels
import random

# TODO: Implement the training loop
# HINT: Build upon what we gave you for HW2.
# See cell below for how we call this training loop.

def training_loop(
    num_epochs,
    batch_size,
    train_features,
    train_labels,
    dev_features,
    dev_labels,
    optimizer,
    model,
    labels2i,
    pad_feature_idx
):
    # TODO: Zip the train features and labels
    
    # TODO: Randomize them, while keeping them paired.
    
    # TODO: Build batches
    samples = list(zip(train_features, train_labels))
    random.shuffle(samples)
    batches = []
    for i in range(0, len(samples), batch_size):
        batches.append(samples[i:i+batch_size])
    print("Training...")
    for i in range(num_epochs):
        losses = []
        for batch in tqdm(batches):
            # Here we get the features and labels, pad them,
            # and build a mask so that our model ignores PADs
            # We have abstracted the padding from you for simplicity, 
            # but please reach out if you'd like learn more.
            features, labels = zip(*batch)
            features = pad_features(features, pad_feature_idx)
            features = torch.stack(features)
            # Pad the label sequences to all be the same size, so we
            # can form a proper matrix.
            labels = pad_labels(labels, labels2i[PAD_SYMBOL])
            labels = torch.stack(labels)
            mask = (labels != labels2i[PAD_SYMBOL])
            # TODO: Empty the dynamic computation graph
            optimizer.zero_grad()
            # TODO: Run the model. Since we use the pytorch-crf model,
            # our forward function returns the positive log-likelihood already.
            # We want the negative log-likelihood. See crf.py forward method in NERTagger
            loss = -1 * model.forward(features, labels, mask)
            # TODO: Backpropogate the loss through our model
            loss.backward()
            # TODO: Update our coefficients in the direction of the gradient.
            optimizer.step()
            # TODO: Store the losses for logging
            losses.append(loss.item())
        
        # TODO: Log the average Loss for the epoch
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # TODO: make dev predictions with the `predict()` function
        dev_predictions = predict(model, dev_features)
        # TODO: Compute F1 score on the dev set and log it.
        dev_f1 = f1_score(dev_predictions, dev_labels, labels2i['O'])
        print(f'F1: {dev_f1}')
        
    # Return the trained model
    return model

## Run the training loop   (10 Points)

We have provided the code here, but you can try different hyperparameters and test multiple runs.

In [14]:
from crf import build_features_set
from crf import make_features_dict
from crf import encode_features, encode_labels
from crf import NERTagger

# Build the model and featurized data
train_features = featurize(train_sents)
dev_features = featurize(dev_sents)

# Get the full inventory of possible features
all_features = build_features_set(train_features)
# Hash all features to a unique int.
features_dict = make_features_dict(all_features)

Building features set!


100%|█████████████████████████████████████████████████████████████████████████████| 1404/1404 [00:00<00:00, 9217.11it/s]

Found 74582 features


In [15]:
encoded_train_features = encode_features(train_features, features_dict)
encoded_dev_features = encode_features(dev_features, features_dict)
encoded_train_labels = encode_labels(train_tag_sents, labels2i)
encoded_dev_labels = encode_labels(dev_tag_sents, labels2i)

In [16]:
for feat, label, sents in zip(dev_features, dev_tag_sents, dev_sents):
    if len(feat) != len(label):
        print(len(feat), len(label), len(sents))
        # print(len(sents), len(feat))
        print(sents)
        print([x[0] for x in feat])

In [17]:
len(encoded_dev_features[0])

354

In [20]:
# TODO: Play with hyperparameters here.

# Initialize the model.
model = NERTagger(len(features_dict), len(labels2i))

num_epochs = 100
batch_size = 16
LR=0.009
optimizer = torch.optim.SGD(model.parameters(), LR)

model = training_loop(
    num_epochs,
    batch_size,
    encoded_train_features,
    encoded_train_labels,
    encoded_dev_features,
    encoded_dev_labels,
    optimizer,
    model,
    labels2i,
    features_dict[PAD_SYMBOL]
)

Training...


100%|███████████████████████████████████████████████████████████████████████████████████| 88/88 [00:41<00:00,  2.12it/s]


epoch 0, loss: 218.72774895754728
F1: tensor([0.0304])


100%|███████████████████████████████████████████████████████████████████████████████████| 88/88 [00:41<00:00,  2.11it/s]


epoch 1, loss: 126.92378442937678
F1: tensor([0.1180])


...

epoch 97, loss: 22.660954854705118
F1: tensor([0.5771])


100%|███████████████████████████████████████████████████████████████████████████████████| 88/88 [00:32<00:00,  2.69it/s]


epoch 98, loss: 22.578356981277466
F1: tensor([0.5787])


100%|███████████████████████████████████████████████████████████████████████████████████| 88/88 [00:32<00:00,  2.67it/s]


epoch 99, loss: 22.457206574353304
F1: tensor([0.5792])


In [44]:
def class_precision(
    predicted_labels: List[torch.Tensor],
    true_labels: List[torch.Tensor],
    b_tag_idx: int,
    i_tag_idx: int,
):
    """
    Precision is True Positives / All Positives Predictions
    """
    TP = torch.tensor([0])
    denom = torch.tensor([0])
    for pred, true in zip(predicted_labels, true_labels):
        TP += sum((pred == true)[pred == b_tag_idx]) + sum((pred == true)[pred == i_tag_idx])
        denom += sum(pred == b_tag_idx) + sum(pred == i_tag_idx)

    # Avoid division by 0
    denom = torch.tensor(1) if denom == 0 else denom
    return TP / denom


def class_recall(
    predicted_labels: List[torch.Tensor],
    true_labels: List[torch.Tensor],
    b_tag_idx: int,
    i_tag_idx: int,
):
    """
    Recall is True Positives / All Positive Labels
    """
    TP = torch.tensor([0])
    denom = torch.tensor([0])
    for pred, true in zip(predicted_labels, true_labels):
        TP += sum((pred == true)[true == b_tag_idx]) + sum((pred == true)[true == i_tag_idx])
        denom += sum(true == b_tag_idx) + sum(true == i_tag_idx)

    # Avoid division by 0
    denom = torch.tensor(1) if denom == 0 else denom
    return TP / denom


def class_f1_score(predicted_labels, true_labels, b_tag_idx, i_tag_idx):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = class_precision(predicted_labels, true_labels, b_tag_idx, i_tag_idx)
    R = class_recall(predicted_labels, true_labels, b_tag_idx, i_tag_idx)
    return 2*P*R/(P+R)

In [32]:
dev_predictions = predict(model, encoded_dev_features)

In [39]:
class_precision(dev_predictions, encoded_dev_labels, labels2i['B-OTHER_PERSON'], labels2i['I-OTHER_PERSON'])

tensor([0.])

In [47]:
check_tags = ["COURT", "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER"]
f1_scores = []
for tag in check_tags:
    f1 = class_f1_score(dev_predictions, encoded_dev_labels, labels2i[f'B-{tag}'], labels2i[f'I-{tag}'])
    print(f"{tag}: {f1}")
    f1_scores.append(f1)

COURT: tensor([0.7109])
PETITIONER: tensor([0.4163])
RESPONDENT: tensor([0.5166])
JUDGE: tensor([0.6693])
LAWYER: tensor([0.7289])


In [48]:
sum(f1_scores) / len(f1_scores)

tensor([0.6084])